In [1]:
# Import the proper libraries we need for this

import os
import json
import uuid
import hashlib
import datetime
from typing import Dict, List, Optional, TypedDict, Literal
from dataclasses import dataclass, asdict, field
import getpass
from IPython.display import clear_output


In [2]:
# Define domain models with TypeScript-like interfaces since I'm comfortable developing in TypeScript

TaskStatus = Literal["Pending", "Completed"]

class TaskDict(TypedDict):
    id: str
    description: str
    status: TaskStatus
    created_at: str

@dataclass
class Task:
    description: str
    status: TaskStatus = "Pending"
    id: str = field(default_factory=lambda: str(uuid.uuid4()))
    created_at: str = field(default_factory=lambda: datetime.datetime.now().isoformat())
    
    def to_dict(self) -> TaskDict:
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: TaskDict) -> 'Task':
        return cls(
            description=data["description"],
            status=data["status"],
            id=data["id"],
            created_at=data["created_at"]
        )

class UserDict(TypedDict):
    username: str
    password_hash: str

@dataclass
class User:
    username: str
    password_hash: str
    
    def to_dict(self) -> UserDict:
        return asdict(self)
    
    @classmethod
    def from_dict(cls, data: UserDict) -> 'User':
        return cls(
            username=data["username"],
            password_hash=data["password_hash"]
        )
    
    @classmethod
    def create(cls, username: str, password: str) -> 'User':
        """Create a new user with a hashed password"""
        password_hash = cls.hash_password(password)
        return cls(username=username, password_hash=password_hash)
    
    @staticmethod
    def hash_password(password: str) -> str:
        """Hash a password using SHA-256"""
        return hashlib.sha256(password.encode()).hexdigest()
    
    def verify_password(self, password: str) -> bool:
        """Verify if the provided password matches the stored hash"""
        return self.password_hash == self.hash_password(password)
        


In [3]:
# Repository classes for data persistence

class UserRepository:
    def __init__(self, file_path: str = "users.json"):
        self.file_path = file_path
        self._ensure_file_exists()
    
    def _ensure_file_exists(self) -> None:
        """Create the users file if it doesn't exist"""
        if not os.path.exists(self.file_path):
            with open(self.file_path, 'w') as f:
                json.dump([], f)
    
    def get_all(self) -> List[User]:
        """Get all users from the file"""
        with open(self.file_path, 'r') as f:
            data = json.load(f)
            return [User.from_dict(user_dict) for user_dict in data]
    
    def get_by_username(self, username: str) -> Optional[User]:
        """Find a user by username"""
        users = self.get_all()
        for user in users:
            if user.username == username:
                return user
        return None
    
    def save(self, user: User) -> None:
        """Save a new user to the file"""
        users = self.get_all()
        # Remove user if already exists
        users = [u for u in users if u.username != user.username]
        # Add the new/updated user
        users.append(user)
        # Save to file
        with open(self.file_path, 'w') as f:
            json.dump([u.to_dict() for u in users], f, indent=2)

class TaskRepository:
    def __init__(self, username: str):
        self.file_path = f"tasks_{username}.json"
        self._ensure_file_exists()
    
    def _ensure_file_exists(self) -> None:
        """Create the tasks file if it doesn't exist"""
        if not os.path.exists(self.file_path):
            with open(self.file_path, 'w') as f:
                json.dump([], f)
    
    def get_all(self) -> List[Task]:
        """Get all tasks from the file"""
        with open(self.file_path, 'r') as f:
            data = json.load(f)
            return [Task.from_dict(task_dict) for task_dict in data]
    
    def get_by_id(self, task_id: str) -> Optional[Task]:
        """Find a task by ID"""
        tasks = self.get_all()
        for task in tasks:
            if task.id == task_id:
                return task
        return None
    
    def save(self, task: Task) -> None:
        """Save a new or updated task to the file"""
        tasks = self.get_all()
        # Remove task if it already exists (for updates)
        tasks = [t for t in tasks if t.id != task.id]
        # Add the new/updated task
        tasks.append(task)
        # Save to file
        with open(self.file_path, 'w') as f:
            json.dump([t.to_dict() for t in tasks], f, indent=2)
    
    def delete(self, task_id: str) -> bool:
        """Delete a task by ID"""
        tasks = self.get_all()
        initial_count = len(tasks)
        tasks = [t for t in tasks if t.id != task_id]
        
        # Only save if a task was actually removed
        if len(tasks) < initial_count:
            with open(self.file_path, 'w') as f:
                json.dump([t.to_dict() for t in tasks], f, indent=2)
            return True
        return False
        

In [4]:
# Service classes for business logic

class AuthenticationService:
    def __init__(self, user_repository: UserRepository):
        self.user_repository = user_repository
        self.current_user: Optional[User] = None
    
    def register(self, username: str, password: str) -> bool:
        """Register a new user"""
        # Check if username already exists
        if self.user_repository.get_by_username(username):
            return False
        
        # Create and save the new user
        user = User.create(username, password)
        self.user_repository.save(user)
        return True
    
    def login(self, username: str, password: str) -> bool:
        """Login a user"""
        user = self.user_repository.get_by_username(username)
        if user and user.verify_password(password):
            self.current_user = user
            return True
        return False
    
    def logout(self) -> None:
        """Logout the current user"""
        self.current_user = None
    
    @property
    def is_authenticated(self) -> bool:
        """Check if a user is currently authenticated"""
        return self.current_user is not None
    
    @property
    def username(self) -> Optional[str]:
        """Get the current username or None if not authenticated"""
        return self.current_user.username if self.current_user else None

class TaskService:
    def __init__(self, task_repository: TaskRepository):
        self.task_repository = task_repository
    
    def add_task(self, description: str) -> Task:
        """Add a new task"""
        task = Task(description=description)
        self.task_repository.save(task)
        return task
    
    def get_all_tasks(self) -> List[Task]:
        """Get all tasks"""
        return self.task_repository.get_all()
    
    def complete_task(self, task_id: str) -> bool:
        """Mark a task as completed"""
        task = self.task_repository.get_by_id(task_id)
        if task:
            task.status = "Completed"
            self.task_repository.save(task)
            return True
        return False
    
    def delete_task(self, task_id: str) -> bool:
        """Delete a task"""
        return self.task_repository.delete(task_id)


In [5]:
# User Interface

class ConsoleUI:
    def __init__(self, auth_service: AuthenticationService):
        self.auth_service = auth_service
        self.task_service: Optional[TaskService] = None
    
    def clear_screen(self) -> None:
        """Clear the console screen"""
        clear_output(wait=True)
    
    def print_header(self, title: str) -> None:
        """Print a header with the given title"""
        self.clear_screen()
        print("=" * 50)
        print(f"{title:^50}")
        print("=" * 50)
        print()
    
    def authenticate(self) -> bool:
        """Handle user authentication (login or register)"""
        while True:
            self.print_header("Task Manager - Authentication")
            print("1. Login")
            print("2. Register")
            print("3. Exit")
            choice = input("\nEnter your choice (1-3): ").strip()
            
            if choice == "1":
                return self.login()
            elif choice == "2":
                self.register()
            elif choice == "3":
                return False
            else:
                input("Invalid choice. Press Enter to try again...")
    
    def login(self) -> bool:
        """Handle user login"""
        self.print_header("Login")
        username = input("Username: ").strip()
        # Use getpass to hide password input if not in Jupyter
        # In Jupyter, this will still work but password will be visible
        password = getpass.getpass("Password: ")
        
        if self.auth_service.login(username, password):
            # Initialize task service for the logged-in user
            self.task_service = TaskService(TaskRepository(username))
            return True
        else:
            input("Invalid username or password. Press Enter to continue...")
            return False
    
    def register(self) -> None:
        """Handle user registration"""
        self.print_header("Register")
        username = input("Username: ").strip()
        password = getpass.getpass("Password: ")
        confirm_password = getpass.getpass("Confirm Password: ")
        
        if password != confirm_password:
            input("Passwords do not match. Press Enter to continue...")
            return
        
        if self.auth_service.register(username, password):
            input("Registration successful! Press Enter to continue...")
        else:
            input("Username already exists. Press Enter to continue...")
    
    def show_tasks(self) -> None:
        """Display all tasks for the current user"""
        if not self.task_service:
            return
        
        tasks = self.task_service.get_all_tasks()
        self.print_header(f"Tasks for {self.auth_service.username}")
        
        if not tasks:
            print("No tasks found.")
            input("\nPress Enter to continue...")
            return
        
        # Display tasks in a tabular format
        print(f"{'ID':<36} | {'Description':<30} | {'Status':<10}")
        print("-" * 80)
        
        for task in tasks:
            print(f"{task.id:<36} | {task.description:<30} | {task.status:<10}")
        
        input("\nPress Enter to continue...")
    
    def add_task(self) -> None:
        """Add a new task"""
        if not self.task_service:
            return
        
        self.print_header("Add Task")
        description = input("Enter task description: ").strip()
        
        if not description:
            input("Task description cannot be empty. Press Enter to continue...")
            return
        
        self.task_service.add_task(description)
        input("Task added successfully! Press Enter to continue...")
    
    def complete_task(self) -> None:
        """Mark a task as completed"""
        if not self.task_service:
            return
        
        self.print_header("Complete Task")
        tasks = self.task_service.get_all_tasks()
        pending_tasks = [t for t in tasks if t.status == "Pending"]
        
        if not pending_tasks:
            input("No pending tasks found. Press Enter to continue...")
            return
        
        # Display pending tasks
        print("Pending Tasks:")
        print(f"{'#':<3} | {'ID':<36} | {'Description':<30}")
        print("-" * 75)
        
        for i, task in enumerate(pending_tasks, 1):
            print(f"{i:<3} | {task.id:<36} | {task.description:<30}")
        
        try:
            task_num = int(input("\nEnter task number to complete (or 0 to cancel): "))
            if task_num == 0:
                return
            
            task = pending_tasks[task_num - 1]
            if self.task_service.complete_task(task.id):
                input("Task marked as completed! Press Enter to continue...")
            else:
                input("Failed to complete task. Press Enter to continue...")
        except (ValueError, IndexError):
            input("Invalid selection. Press Enter to continue...")
    
    def delete_task(self) -> None:
        """Delete a task"""
        if not self.task_service:
            return
        
        self.print_header("Delete Task")
        tasks = self.task_service.get_all_tasks()
        
        if not tasks:
            input("No tasks found. Press Enter to continue...")
            return
        
        # Display all tasks
        print("All Tasks:")
        print(f"{'#':<3} | {'ID':<36} | {'Description':<30} | {'Status':<10}")
        print("-" * 85)
        
        for i, task in enumerate(tasks, 1):
            print(f"{i:<3} | {task.id:<36} | {task.description:<30} | {task.status:<10}")
        
        try:
            task_num = int(input("\nEnter task number to delete (or 0 to cancel): "))
            if task_num == 0:
                return
            
            task = tasks[task_num - 1]
            confirm = input(f"Are you sure you want to delete task '{task.description}'? (y/n): ").lower()
            
            if confirm == 'y':
                if self.task_service.delete_task(task.id):
                    input("Task deleted successfully! Press Enter to continue...")
                else:
                    input("Failed to delete task. Press Enter to continue...")
        except (ValueError, IndexError):
            input("Invalid selection. Press Enter to continue...")
    
    def main_menu(self) -> None:
        """Display the main menu and handle user choices"""
        while self.auth_service.is_authenticated:
            self.print_header(f"Task Manager - Welcome {self.auth_service.username}")
            print("1. Add Task")
            print("2. View Tasks")
            print("3. Mark Task as Completed")
            print("4. Delete Task")
            print("5. Logout")
            
            choice = input("\nEnter your choice (1-5): ").strip()
            
            if choice == "1":
                self.add_task()
            elif choice == "2":
                self.show_tasks()
            elif choice == "3":
                self.complete_task()
            elif choice == "4":
                self.delete_task()
            elif choice == "5":
                self.auth_service.logout()
                self.task_service = None
                input("Logged out successfully. Press Enter to continue...")
            else:
                input("Invalid choice. Press Enter to try again...")
    
    def run(self) -> None:
        """Run the main application loop"""
        while True:
            if not self.auth_service.is_authenticated:
                if not self.authenticate():
                    break
            self.main_menu()
        
        self.print_header("Thank you for using Task Manager!")
        print("Goodbye!")
        

In [6]:
# Application entry point
def run_task_manager():
    user_repo = UserRepository()
    auth_service = AuthenticationService(user_repo)
    app = ConsoleUI(auth_service)
    app.run()
    
    

In [ ]:
# Run the application
run_task_manager()


          Task Manager - Authentication           

1. Login
2. Register
3. Exit


In [ ]:
2